In [1]:
from scipy.io import loadmat
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import numpy as np 
from scipy.optimize import minimize
import networkx as nx
import scipy.linalg as slin
import scipy.optimize as sopt
from scipy.special import expit as sigmoid
import warnings
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Union

from package.utils import opt_boundary
from package.pdbn import admm_dy_personal as adp

In [2]:
# Load the .mat file
mat_data = loadmat('sims/sim3.mat')  # Replace 'your_file.mat' with your file name
net = mat_data['net'] 
ts = mat_data['ts']   

print(net.shape)

Nsubjects = 50
timepoints_per_subject = 200
subjects_ts = np.split(ts, Nsubjects)


(50, 15, 15)


In [3]:
def create_lagged_version(X, lag):
    """
    Create a time-lagged version of the matrix X based on the number of lagged blocks.

    Args:
        X (np.ndarray): Original time-series data matrix of shape (n, d).
        lag (int): Number of lags.

    Returns:
        Y (np.ndarray): Lagged version of X with multiple lags.
    """
    n, d = X.shape
    Y = np.zeros((n, lag * d))

    for l in range(1, lag + 1):
        Y[l:, (l - 1) * d:l * d] = X[:-l, :]  

    return Y


def Unwe_G(weighted_matrix):

    unweighted_matrix = (weighted_matrix != 0).astype(int)
    
    return unweighted_matrix

In [4]:
# Choose a subject ID 
subject_id = [0, 2, 4, 6, 8]
X_agent = []
Y_agent = []
GT = []

# Extract the adjacency matrix for the selected subject
for k in subject_id:

  gt = net[k]
  ts = subjects_ts[k]

  X = ts
  lag = 1
  Y = create_lagged_version(X, lag)
  GT.append(gt)
  X_agent.append(X)
  Y_agent.append(Y)

X_agent = np.stack(X_agent)  
Y_agent = np.stack(Y_agent)  
GT = np.stack(GT) 

print(X_agent.shape)
print(Y_agent.shape)

(5, 200, 15)
(5, 200, 15)


In [5]:
X_agent = np.array(X_agent, dtype=np.float64)
Y_agent = np.array(Y_agent, dtype=np.float64)
d = 15

bnds = opt_boundary(X,Y,d)

mu = 0.1 
w_threshold = 0.2
a_threshold = 0.2
lambda_w = 0.01
lambda_a = 0.01

In [6]:
_, _, W_k, A_k=  adp(X_agent, Y_agent, bnds, mu = mu, lambda_w=lambda_w, lambda_a=lambda_a, w_threshold=w_threshold, a_threshold=a_threshold)

In [7]:
from sklearn.metrics import roc_auc_score

AUROC = np.zeros(5)

for k in range(5):

    # sum over W and A ele
    final = W_k[k] + A_k[k]
    threshold = 0.2

    final[np.abs(final) < threshold] = 0
    AUROC[k] = roc_auc_score(Unwe_G(GT[k]).flatten(), Unwe_G(final).flatten())

print(AUROC)
print(np.mean(AUROC))
print(np.std(AUROC))

[0.66524621 0.76704545 0.70123106 0.77698864 0.77225379]
0.7365530303030303
0.045103601026170906
